<a href="https://colab.research.google.com/github/DigiBP/digibp-camunda-external-python-task/blob/master/external_python_worker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# External Python Worker

An external task client allows to set up remote service tasks for a workflow. This example shows the external task execution by a Python worker.

![alt text](https://raw.githubusercontent.com/DigiBP/digibp-camunda-external-python-task/master/modelling/pizza-menu-external-demo-decision.png)
In this example, we are going to consume the REST API of the Camunda Workflow engine and make use of HTTP Long Polling. Specifically, the following REST endpoints are used:


```
POST /external-task/fetchAndLock
{
   "workerId":"workerid",
   "maxTasks":1,
   "usePriority":true,
   "asyncResponseTimeout":29000,
   "topics":[
      {
         "topicName":"GetSurpriseMenu",
         "lockDuration":20000
      }
   ]
}

POST /external-task/{id}/complete
{
   "workerId":"workerid",
   "variables":{
      "menu":{
         "value":"pizza"
      }
   }
}
```


The use of the REST endpoint is encapsulated within the cam module of this GitHub repository.

You can import the *cam* module from GitHub Repository with [httpimport](https://github.com/operatorequals/httpimport).

In [1]:
import httpimport
with httpimport.github_repo('DigiBP', 'digibp-camunda-external-python-task', 'cam'): import cam

## 0 Initialize Client Worker
Initialize the client worker as follows:

In [2]:
worker = cam.Client("https://digibp.herokuapp.com/engine-rest")

## 1 Procedural Variant

### Subscription

First, we subscribe tasks of the corresponding topic *GetSurpriseMenu* (with tenant ID *showcase* (optional)):

In [ ]:
response = worker.subscribe("GetSurpriseMenu", None, "showcase")

Then we need the current taskid for correlating it during completion:![alt text](https://)

In [ ]:
taskid = str(response[0]['id'])

We may be getting some workflow data/variables:

In [ ]:
# vegetarian_guests = body[0]['variables']['vegetarian']['value']

Then we can define a dictionary of workflow data/variables if required:

In [ ]:
variables = {"menu": "pizza"}

And finally, we complete the task and transfer some workflow variables:

In [ ]:
worker.complete(taskid, **variables)

## 2 Concurrent Variant

Define a callback function:

In [3]:
import random


def get_surprise_menu_callback(taskid, response):
    try:
        vegetarian_guests = response[0]['variables']['vegetarian']['value']
    except:
        vegetarian_guests = False

    if vegetarian_guests:
        menu = random.choice(["pizza", "pasta", "verdura"])
    else:
        menu = random.choice(["pizza", "pasta", "carne", "verdura"])

    variables = {"menu": menu}
    worker.complete(taskid, **variables)

Subscribe the topic *GetSurpriseMenu* (with tenant ID *showcase* (optional)) and assign the callback function:

In [8]:
worker.subscribe("GetSurpriseMenu", get_surprise_menu_callback, "showcase")

Poll new tasks of the subscribed topics:

In [9]:
worker.polling()

polling subscription: GetSurpriseMenu
